# Mẫu 01: Trò chuyện nhanh qua OpenAI SDK

Notebook này minh họa cách sử dụng OpenAI SDK với Microsoft Foundry Local để thực hiện các tương tác trò chuyện nhanh.

## Tổng quan

Ví dụ này cho thấy:
- Sử dụng OpenAI Python SDK với Foundry Local
- Xử lý cả cấu hình Azure OpenAI và Foundry Local
- Triển khai xử lý lỗi đúng cách và chiến lược dự phòng
- Sử dụng FoundryLocalManager để quản lý dịch vụ


## Yêu cầu trước

Hãy đảm bảo bạn đã:
1. Cài đặt và chạy Foundry Local
2. Tải một mô hình (ví dụ: `foundry model run phi-4-mini`)
3. Cài đặt các gói Python cần thiết

### Cài đặt các phụ thuộc


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## Nhập Các Thư Viện Cần Thiết


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## Cấu hình

Thiết lập cấu hình của bạn. Bạn có thể sử dụng một trong hai:
1. **Azure OpenAI** (dựa trên đám mây)
2. **Foundry Local** (suy luận cục bộ)

### Lựa chọn 1: Cấu hình Azure OpenAI


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### Tùy chọn 2: Cấu hình Cục bộ Foundry


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## Khởi tạo Client

Tạo client OpenAI dựa trên cấu hình của bạn:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## Ví dụ Chat Cơ Bản

Hãy thử một tương tác chat đơn giản:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## Phiên trò chuyện tương tác

Thử các loại lời nhắc khác nhau để xem cách mô hình phản hồi:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## Sử dụng nâng cao: Phản hồi dạng streaming

Đối với các phản hồi dài, streaming có thể mang lại trải nghiệm người dùng tốt hơn:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## Xử lý lỗi và chẩn đoán

Hãy kiểm tra trạng thái dịch vụ và các mô hình có sẵn:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## Kiểm Tra Lời Nhắc Tùy Chỉnh

Sử dụng ô dưới đây để kiểm tra các lời nhắc của bạn:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## Tóm tắt

Notebook này đã trình bày:

1. **✅ Cài đặt Client**: Cách cấu hình OpenAI SDK với Foundry Local
2. **✅ Chat cơ bản**: Tương tác yêu cầu-phản hồi đơn giản
3. **✅ Streaming**: Tạo phản hồi theo thời gian thực
4. **✅ Xử lý lỗi**: Xử lý lỗi mạnh mẽ và chẩn đoán
5. **✅ Tình trạng dịch vụ**: Kiểm tra khả dụng của mô hình và trạng thái dịch vụ

### Bước tiếp theo

- **Sample 02**: Tích hợp SDK nâng cao với hỗ trợ Azure OpenAI
- **Sample 04**: Xây dựng ứng dụng chat Chainlit
- **Sample 05**: Hệ thống điều phối đa tác nhân
- **Sample 06**: Định tuyến mô hình thông minh

### Lợi ích chính của Foundry Local

- 🔒 **Bảo mật**: Dữ liệu không bao giờ rời khỏi thiết bị của bạn
- ⚡ **Tốc độ**: Suy luận cục bộ với độ trễ thấp
- 💰 **Chi phí**: Không có chi phí sử dụng API
- 🔌 **Ngoại tuyến**: Hoạt động mà không cần kết nối internet
- 🛠️ **Tương thích**: API tương thích với OpenAI
